## Implementing the Logistic Regression as Neural network 

In [21]:
import numpy as np
import h5py

In [23]:
def load_dataset():
    train_dataset = h5py.File('Cat vs dog dataset/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('Cat vs dog dataset/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [25]:
X_train,y_train,X_test,y_test,classes = load_dataset()

# Pytorch Implementation

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [30]:
# Converting Numpy arrays to torch tensors data type
X_train_flat = X_train.reshape(X_train.shape[0], -1) / 255.
X_test_flat = X_test.reshape(X_test.shape[0], -1) / 255.
y_train_T = y_train.T
y_test_T = y_test.T

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_flat, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_T, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_flat, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_T, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [32]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [34]:
model = LogisticRegressionModel(X_train_tensor.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [36]:
epochs = 1000
for epoch in range(epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs).squeeze()
        labels = labels.squeeze()
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 41.17647171020508
Epoch 100, Loss: 35.29411697387695
Epoch 200, Loss: 23.52941131591797
Epoch 300, Loss: 47.05882263183594
Epoch 400, Loss: 35.29411697387695
Epoch 500, Loss: 29.41176414489746
Epoch 600, Loss: 29.41176414489746
Epoch 700, Loss: 29.41176414489746
Epoch 800, Loss: 52.94117736816406
Epoch 900, Loss: 35.29411697387695


In [38]:
# Prediction and Accuracy
with torch.no_grad():
    preds = (model(X_test_tensor).squeeze() > 0.5).float()
    acc = (preds == y_test_tensor.squeeze()).float().mean()
    print(f"Test Accuracy: {acc.item() * 100:.2f}%")

Test Accuracy: 34.00%


In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [42]:
X_train_flat = X_train.reshape(X_train.shape[0], -1) / 255.
X_test_flat = X_test.reshape(X_test.shape[0], -1) / 255.
y_train_T = y_train.T
y_test_T = y_test.T

In [44]:
# Model
model = Sequential([
    Dense(1, activation='sigmoid', input_shape=(X_train_flat.shape[1],))
])

model.compile(optimizer=SGD(learning_rate=0.5), loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
# Training
model.fit(X_train_flat, y_train_T, epochs=1000, batch_size=64, verbose=0)

# Evaluation
loss, accuracy = model.evaluate(X_test_flat, y_test_T, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 64.00%
